импорт библиотек

In [1]:
import numpy as np
import pandas as pd

чтение обучающих данных

In [2]:
trainsent = pd.read_csv(
    "simplesentiment/products_sentiment_train.tsv",
    sep = '\t',
    header = None)
trainsent.columns = ["text", "feat"]

In [3]:
texts = trainsent.text.values
labels = trainsent.feat.values

импорт класса счетчика

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

инициализация счетчика, стандартные стоп-слова, ограничу редкие слова двумя документами

In [5]:
cv = CountVectorizer(stop_words='english', min_df=2)

расчет частотной таблицы

In [6]:
word_count_vector = cv.fit_transform(texts)

In [8]:
featnames = cv.get_feature_names()

In [9]:
word_count_vector.shape, len(featnames)

((2000, 1716), 1716)

смотрю, какие слова самые распространенные

In [10]:
df = pd.DataFrame(
    np.transpose(np.sum(word_count_vector, axis = 0)),
    index = featnames,
    columns = ["counts"]
)

In [11]:
df.sort_values(["counts"], ascending=False).head(7)

,counts
camera,168
phone,161
player,154
great,153
use,152
good,127
quality,123


Самые распространенные три слова не несут никакой информационной нагрузки, исключу-ка я их.
На **stackoverflow** нашелся следующий способ обновить стоп-слова:

In [12]:
from sklearn.feature_extraction import text 

In [13]:
my_additional_stop_words = [u'camera', u'phone', u'player']
stopwords = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

Заново объявляю класс счетчика

In [14]:
cv = CountVectorizer(stop_words=stopwords, min_df=2)

In [15]:
word_count_vector = cv.fit_transform(texts)
featnames = cv.get_feature_names()
word_count_vector.shape, len(featnames)

((2000, 1713), 1713)

Что по документам? Просуммирую частоты в каждом

In [16]:
df = pd.DataFrame(
    np.sum(word_count_vector, axis = 1),
    index = texts,
    columns = ["wordsums"]
)

In [17]:
df = trainsent.copy()

In [18]:
df['wordsum'] = np.sum(word_count_vector, axis = 1)

In [19]:
np.sum(df['wordsum'] == 0)

4

В четырех докуменгтах ни одного значащего слова.
Возьмем признаки не по словам, а по частям слов:

In [88]:
cv = CountVectorizer(stop_words=stopwords, min_df=4, ngram_range=(2,5), analyzer='char_wb')

In [89]:
word_count_vector = cv.fit_transform(texts)
featnames = cv.get_feature_names()
word_count_vector.shape, len(featnames)

((2000, 10390), 10390)

In [29]:
df = pd.DataFrame(
    np.sum(word_count_vector, axis = 1),
    index = texts,
    columns = ["wordsums"]
)

In [30]:
df = trainsent.copy()

In [31]:
df['wordsum'] = np.sum(word_count_vector, axis = 1)

In [32]:
df.shape

(2000, 3)

In [33]:
np.sum(df['wordsum'] == 0)

0

С таким значение **min_df** пустых документов нет

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

Возьму несколько классификаторов

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

И по очереди смотрю: логистическая регрессия

In [52]:
%%time
lgr = LogisticRegression()
lgr.fit(word_count_vector,labels)
score_lgr = cross_val_score(lgr, word_count_vector, labels, scoring='roc_auc', cv=5)

CPU times: user 2.47 s, sys: 116 ms, total: 2.59 s
Wall time: 1.31 s


In [53]:
score_lgr.min(), score_lgr.mean()

(0.7973225152129817, 0.8151138423821394)

Случайный лес:

In [41]:
%%time
ranf = RandomForestClassifier(n_estimators=1000)
ranf.fit(word_count_vector,labels)
score_ranf = cross_val_score(ranf, word_count_vector, labels, scoring='roc_auc', cv=5)

CPU times: user 1min 27s, sys: 39.9 ms, total: 1min 27s
Wall time: 1min 27s


In [43]:
score_ranf.min(), score_ranf.mean()

(0.7900886381950041, 0.8339786077825325)

Градиент-бустинг:

In [54]:
%%time
grab = GradientBoostingClassifier(n_estimators=1000)
grab.fit(word_count_vector,labels)
score_grab = cross_val_score(grab, word_count_vector, labels, scoring='roc_auc', cv=5)

CPU times: user 3min 13s, sys: 116 ms, total: 3min 13s
Wall time: 3min 13s


In [55]:
score_grab.min(), score_grab.mean()

(0.8339418526031102, 0.8447339381696463)

Возьму ансамблирование этих трех классификаторов

In [56]:
from sklearn.ensemble import VotingClassifier

In [65]:
%%time
eclf = VotingClassifier(estimators=[('lgr', lgr), ('ranf', ranf), ('grab', grab)], voting='soft')
eclf.fit(word_count_vector,labels)
score_eclf = cross_val_score(eclf, word_count_vector, labels, scoring='roc_auc', cv=5)

CPU times: user 4min 29s, sys: 288 ms, total: 4min 30s
Wall time: 4min 27s


In [67]:
score_eclf.min(), score_eclf.mean()

(0.848762677484787, 0.8535082380450311)

Считываю тестовый датасет

In [59]:
testsent = pd.read_csv("simplesentiment/products_sentiment_test.tsv", sep = '\t', index_col='Id')

Пайплайн делать лениво, всего два элемента, зачем?

In [92]:
test_cnt = cv.transform(testsent['text'].values)
lab_predict = eclf.predict(test_cnt)

Сохраняю результат

In [101]:
df_lab = pd.DataFrame(lab_predict, columns=['y'])

In [104]:
df_lab.head()

,y
0,1
1,0
2,1
3,1
4,0


In [103]:
df_lab.index.name

In [105]:
df_lab.to_csv("out.csv")